In [1]:
!pip install --upgrade gspread gspread_dataframe oauth2client

In [2]:
!pip install laion-clap

In [3]:
# Authenticate with Google in Colab
from google.colab import auth
auth.authenticate_user()

# Authorize with gspread
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet by URL
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8/edit#gid=1706329183")

worksheet = sh.worksheet("htsat-fused")

In [4]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np
import laion_clap

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [5]:
# Load CLAP model + processor
model = laion_clap.CLAP_Module(enable_fusion=True)
model.load_ckpt()  # download the default pretrained checkpoint.

clapModel = ClapModel.from_pretrained("laion/clap-htsat-fused")
processor = AutoProcessor.from_pretrained("laion/clap-htsat-fused")

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
Download completed!
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branch.patch_embed.fus

config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

# Process audio

In [6]:
import zipfile
import os

zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

extract_dir = "/content/" # You can change this if you want to extract elsewhere
os.makedirs(extract_dir, exist_ok=True)

for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Error: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


In [7]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        #Clap has already been trained on a sample rate of 48,000 so we should use what it knows already
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [8]:
audio_embeddings = model.get_audio_embedding_from_data(x=audio_stimuli, use_tensor=False)

audio_embeddings = torch.tensor(audio_embeddings)
print(audio_embeddings.shape)

/usr/local/lib/python3.11/dist-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


torch.Size([59, 512])


# Process text

In [9]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = [tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = [tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake", "like"]

dimensional_captions_perceived = [tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = [tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['happiness', 'sadness', 'anger', 'tenderness', 'fear']
['happiness', 'sadness', 'anger', 'tenderness', 'fear']
['positive', 'relaxed', 'awake', 'like']
['positive', 'relaxed', 'awake', 'like']


In [10]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
tag_embeds = clapModel.get_text_features(**tag_inputs)

In [11]:
tag_inputs_disc_i = processor(text=discrete_captions_induced, return_tensors="pt", padding=True)
tag_embeds_disc_i = clapModel.get_text_features(**tag_inputs_disc_i)

In [12]:
tag_inputs_dim_p = processor(text=dimensional_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_dim_p = clapModel.get_text_features(**tag_inputs_dim_p)

In [13]:
tag_inputs_disc_p = processor(text=discrete_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_disc_p = clapModel.get_text_features(**tag_inputs_disc_p)

## Load csv files and extract related columns

In [14]:
IDim_path = '/content/Exp1/Data/IDim/'
IDim_response_dfs = []

for file in sorted(os.listdir(IDim_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(IDim_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['positive', 'relaxed', 'awake','like']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDim_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDim_response_dfs:
    master_human_responses_df = pd.concat(IDim_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3835, 4)

Master human responses (first 5 rows):
   positive  relaxed  awake  like
0      3.68     3.78   4.42  3.41
1      5.88     5.98   3.89  5.54
2      6.53     5.59   6.59  6.17
3      6.26     5.71   6.88  6.18
4      2.80     2.62   5.15  1.87



In [15]:
PDim_path = '/content/Exp1/Data/PDim/'
PDim_response_dfs = []

for file in sorted(os.listdir(PDim_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(PDim_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['positive', 'relaxed', 'awake', 'like']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                PDim_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if PDim_response_dfs:
    master_human_responses_df_dim_p = pd.concat(PDim_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_dim_p.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_dim_p.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3953, 4)

Master human responses (first 5 rows):
   positive  relaxed  awake  like
0      2.23     9.00   2.37  1.68
1      6.56     5.64   4.12  7.52
2      5.11     6.23   4.04  6.72
3      7.14     7.67   2.95  7.41
4      1.85     1.77   2.98  1.40



In [16]:
IDisc_path = '/content/Exp1/Data/IDisc/'
IDisc_response_dfs = []

for file in sorted(os.listdir(IDisc_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(IDisc_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ["happiness", "sadness", "anger", "tenderness", "fear"]
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDisc_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDisc_response_dfs:
    master_human_responses_df_disc_i = pd.concat(IDisc_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_disc_i.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_disc_i.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3894, 5)

Master human responses (first 5 rows):
   happiness  sadness  anger  tenderness  fear
0       1.00     6.97   5.01        9.00  7.89
1       1.00     6.00   5.80        6.78  1.00
2       1.00     2.42   5.99        6.44  1.09
3       3.92     6.13   2.25        5.96  1.22
4       1.99     7.92   6.82        5.83  1.39



In [17]:
PDisc_path = '/content/Exp1/Data/PDisc/'
PDisc_response_dfs = []

for file in sorted(os.listdir(PDisc_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(PDisc_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ["happiness", "sadness", "anger", "tenderness", "fear"]
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                PDisc_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if PDisc_response_dfs:
    master_human_responses_df_disc_p = pd.concat(PDisc_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_disc_p.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_disc_p.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3835, 5)

Master human responses (first 5 rows):
   happiness  sadness  anger  tenderness  fear
0       1.32     1.30   1.27        1.25  3.18
1       1.00     1.44   1.76        1.00  1.24
2       1.00     5.87   1.23        1.28  1.56
3       1.92     3.36   4.82        1.20  2.32
4       1.23     2.31   7.10        1.61  2.86



# Prepare features X and targets y

In [18]:
from sklearn.model_selection import train_test_split
"""
For each participant:
1. Takes all 59 audio embeddings
2. Pairs them with that participant's 59 ratings
3. Stacks these pairs for all participants.
"""

# Checks the number of participants is an exact multiple of the number of audio files (ensuring every participant rated every stimuli)
num_participants = len(IDim_response_dfs)
if master_human_responses_df.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

# Audio embeddings generated by CLAP
# Creates one copy of all embeddings per participant
X_list = []
for _ in range(num_participants):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X = np.array(X_list)

# Extract y from the concatenated DataFrame
y = master_human_responses_df[['positive', 'relaxed', 'awake', 'like']].values

print(f"Shape of X (features) after implicit alignment: {X.shape}")
print(f"Shape of y (labels) after implicit alignment: {y.shape}\n")

# Sanity check: X and y must have the same number of rows
# Critical check that we have exactly one set of features per label
if X.shape[0] != y.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
# 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

"""
Example:
Result (for 2 participants × 59 files = 118 total samples):
1. X_train: 94 audio embeddings (80% of 118)
2. y_train: 94 corresponding rating vectors
3. X_test: 24 audio embeddings (20%)
4. y_test: 24 rating vectors
'''

# X_train = combination of audio embeddings stacked on top of eachother
# y_train = combination of participants ratings for an audio file stacked ontop of eachother
# x_test = audio embedding for one audio file only
# y_test = participant rating for one audio file only

# Goal: check if x_test and y_test match

print(f"Training set size (X_train, y_train): {X_train.shape}, {y_train.shape}")
print(f"Testing set size (X_test, y_test): {X_test.shape}, {y_test.shape}\n")

In this example:
y_train = [
    [5.0, 4.0, 6.0],  # P1-A
    [7.0, 6.0, 4.0],   # P1-C
    [2.0, 1.0, 6.0],   # P2-B
    [4.0, 3.0, 5.0],   # P2-A
    [6.0, 7.0, 3.0]    # P2-C
]

X_test = [
    [0.5, 0.6, 0.7, 0.8]  # B.wav (P1)
]  # 1 sample

y_test = [
    [3.0, 2.0, 5.0]  # P1-B
]
- Training:
  X_train[0] = [0.1, 0.2, 0.3, 0.4] → Predict y_train[0] = [5.0, 4.0, 6.0]
"""

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 4)



'\nExample:\nResult (for 2 participants × 59 files = 118 total samples):\n1. X_train: 94 audio embeddings (80% of 118)\n2. y_train: 94 corresponding rating vectors\n3. X_test: 24 audio embeddings (20%)\n4. y_test: 24 rating vectors\n\'\'\'\n\n# X_train = combination of audio embeddings stacked on top of eachother\n# y_train = combination of participants ratings for an audio file stacked ontop of eachother\n# x_test = audio embedding for one audio file only\n# y_test = participant rating for one audio file only\n\n# Goal: check if x_test and y_test match\n\nprint(f"Training set size (X_train, y_train): {X_train.shape}, {y_train.shape}")\nprint(f"Testing set size (X_test, y_test): {X_test.shape}, {y_test.shape}\n")\n\nIn this example:\ny_train = [\n    [5.0, 4.0, 6.0],  # P1-A\n    [7.0, 6.0, 4.0],   # P1-C\n    [2.0, 1.0, 6.0],   # P2-B\n    [4.0, 3.0, 5.0],   # P2-A\n    [6.0, 7.0, 3.0]    # P2-C\n]\n\nX_test = [\n    [0.5, 0.6, 0.7, 0.8]  # B.wav (P1)\n]  # 1 sample\n\ny_test = [\n   

In [19]:
from sklearn.model_selection import train_test_split

num_participants_dim_p = len(PDim_response_dfs)
if master_human_responses_df_dim_p.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_dim_p):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_dim_p = np.array(X_list)

# Extract y from the concatenated DataFrame
y_dim_p = master_human_responses_df_dim_p[['positive', 'relaxed', 'awake', 'like']].values

print(f"Shape of X (features) after implicit alignment: {X_dim_p.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_dim_p.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_dim_p.shape[0] != y_dim_p.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_dim_p, X_test_dim_p, y_train_dim_p, y_test_dim_p = train_test_split(
    X_dim_p, y_dim_p, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_dim_p.shape}, {y_train_dim_p.shape}")
print(f"Testing set size (X_test, y_test): {X_test_dim_p.shape}, {y_test_dim_p.shape}\n")

Shape of X (features) after implicit alignment: (3953, 512)
Shape of y (labels) after implicit alignment: (3953, 4)

Training set size (X_train, y_train): (3953, 512), (3162, 4)
Testing set size (X_test, y_test): (791, 512), (791, 4)



In [20]:
from sklearn.model_selection import train_test_split

num_participants_disc_i = len(IDisc_response_dfs)
if master_human_responses_df_disc_i.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_disc_i):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_disc_i = np.array(X_list)

# Extract y from the concatenated DataFrame
y_disc_i = master_human_responses_df_disc_i[["happiness", "sadness", "anger", "tenderness", "fear"]].values

print(f"Shape of X (features) after implicit alignment: {X_disc_i.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_disc_i.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_disc_i.shape[0] != y_disc_i.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_disc_i, X_test_disc_i, y_train_disc_i, y_test_disc_i = train_test_split(
    X_disc_i, y_disc_i, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_disc_i.shape}, {y_train_disc_i.shape}")
print(f"Testing set size (X_test, y_test): {X_test_disc_i.shape}, {y_test_disc_i.shape}\n")

Shape of X (features) after implicit alignment: (3894, 512)
Shape of y (labels) after implicit alignment: (3894, 5)

Training set size (X_train, y_train): (3894, 512), (3115, 5)
Testing set size (X_test, y_test): (779, 512), (779, 5)



In [21]:
from sklearn.model_selection import train_test_split

num_participants_disc_p = len(PDisc_response_dfs)
if master_human_responses_df_disc_p.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_disc_p):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_disc_p = np.array(X_list)

# Extract y from the concatenated DataFrame
y_disc_p = master_human_responses_df_disc_p[["happiness", "sadness", "anger", "tenderness", "fear"]].values

print(f"Shape of X (features) after implicit alignment: {X_disc_p.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_disc_p.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_disc_p.shape[0] != y_disc_p.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_disc_p, X_test_disc_p, y_train_disc_p, y_test_disc_p = train_test_split(
    X_disc_p, y_disc_p, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_disc_p.shape}, {y_train_disc_p.shape}")
print(f"Testing set size (X_test, y_test): {X_test_disc_p.shape}, {y_test_disc_p.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 5)

Training set size (X_train, y_train): (3835, 512), (3068, 5)
Testing set size (X_test, y_test): (767, 512), (767, 5)



# Train regression head (=MLP, a few projection layers)

In [22]:
from sklearn.neural_network import MLPRegressor

mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor.fit(X_train, y_train)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 13.32292420
Validation score: -5.313512
Iteration 2, loss = 9.58739888
Validation score: -2.584479
Iteration 3, loss = 4.64134987
Validation score: -0.535113
Iteration 4, loss = 2.53979194
Validation score: -0.047074
Iteration 5, loss = 1.95005122
Validation score: 0.084410
Iteration 6, loss = 1.82288388
Validation score: 0.127376
Iteration 7, loss = 1.75004873
Validation score: 0.160028
Iteration 8, loss = 1.69694456
Validation score: 0.181772
Iteration 9, loss = 1.65849073
Validation score: 0.197967
Iteration 10, loss = 1.62707197
Validation score: 0.212589
Iteration 11, loss = 1.59886352
Validation score: 0.223419
Iteration 12, loss = 1.57759589
Validation score: 0.229506
Iteration 13, loss = 1.56093154
Validation score: 0.236477
Iteration 14, loss = 1.54753035
Validation score: 0.239977
Iteration 15, loss = 1.53595644
Validation score: 0.241561
Iteration 16, loss = 1.52773643
Validation score: 0.245955
Iteration 17, loss = 1.52

In [23]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_dim_p = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_dim_p.fit(X_train_dim_p, y_train_dim_p)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 13.18594503
Validation score: -4.064813
Iteration 2, loss = 9.13213563
Validation score: -1.664071
Iteration 3, loss = 4.18738648
Validation score: -0.255770
Iteration 4, loss = 2.47774971
Validation score: 0.087810
Iteration 5, loss = 2.00099777
Validation score: 0.165831
Iteration 6, loss = 1.90172965
Validation score: 0.197759
Iteration 7, loss = 1.83485334
Validation score: 0.212305
Iteration 8, loss = 1.78885489
Validation score: 0.227037
Iteration 9, loss = 1.75300670
Validation score: 0.238963
Iteration 10, loss = 1.72415907
Validation score: 0.249020
Iteration 11, loss = 1.69942027
Validation score: 0.255634
Iteration 12, loss = 1.68268860
Validation score: 0.262900
Iteration 13, loss = 1.67002850
Validation score: 0.266099
Iteration 14, loss = 1.65691579
Validation score: 0.269963
Iteration 15, loss = 1.64696922
Validation score: 0.273027
Iteration 16, loss = 1.64046706
Validation score: 0.277122
Iteration 17, loss = 1.635

In [24]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_disc_i = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_disc_i.fit(X_train_disc_i, y_train_disc_i)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 4.88953762
Validation score: -0.698258
Iteration 2, loss = 3.28423668
Validation score: -0.096338
Iteration 3, loss = 2.38556099
Validation score: 0.004865
Iteration 4, loss = 2.25522016
Validation score: 0.049298
Iteration 5, loss = 2.17252834
Validation score: 0.065284
Iteration 6, loss = 2.12258159
Validation score: 0.083047
Iteration 7, loss = 2.08911168
Validation score: 0.089555
Iteration 8, loss = 2.08043261
Validation score: 0.093544
Iteration 9, loss = 2.07228973
Validation score: 0.097158
Iteration 10, loss = 2.05517018
Validation score: 0.098790
Iteration 11, loss = 2.05194464
Validation score: 0.096425
Iteration 12, loss = 2.04554985
Validation score: 0.098398
Iteration 13, loss = 2.03655211
Validation score: 0.100277
Iteration 14, loss = 2.03446679
Validation score: 0.099421
Iteration 15, loss = 2.02674198
Validation score: 0.099861
Iteration 16, loss = 2.02666161
Validation score: 0.097509
Iteration 17, loss = 2.02413

In [25]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_disc_p = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_disc_p.fit(X_train_disc_p, y_train_disc_p)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 7.01754666
Validation score: -1.134170
Iteration 2, loss = 5.04575268
Validation score: -0.338818
Iteration 3, loss = 3.22873105
Validation score: -0.000094
Iteration 4, loss = 2.74220410
Validation score: 0.061781
Iteration 5, loss = 2.58825355
Validation score: 0.118687
Iteration 6, loss = 2.47425900
Validation score: 0.145665
Iteration 7, loss = 2.40242731
Validation score: 0.164536
Iteration 8, loss = 2.36074350
Validation score: 0.174369
Iteration 9, loss = 2.33195517
Validation score: 0.176185
Iteration 10, loss = 2.31228461
Validation score: 0.181672
Iteration 11, loss = 2.30062002
Validation score: 0.180079
Iteration 12, loss = 2.29061062
Validation score: 0.183726
Iteration 13, loss = 2.28343502
Validation score: 0.183128
Iteration 14, loss = 2.27737912
Validation score: 0.185051
Iteration 15, loss = 2.27335547
Validation score: 0.185108
Iteration 16, loss = 2.27175925
Validation score: 0.186239
Iteration 17, loss = 2.2710

# Evaluate

In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred = mlp_regressor.predict(X_test)

print(f"\nShape of predictions (y_pred): {y_pred.shape}")
print(f"First 5 actual values (y_test):\n{y_test[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error = np.abs((y_test - y_pred) / y_test) * 100
mape = np.mean(absolute_percentage_error)
worksheet.update('E5', [[f"{mape:.2f}%"]])
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%\n")

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
worksheet.update('E2', [[f"{mae:.4f}"]])
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
worksheet.update('E6', [[f"{rmse:.4f}"]])
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Positive', 'Relaxed', 'Awake', 'Like']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test[:, i]) > 1e-6 and np.std(y_pred[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test[:, i], y_pred[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test.shape[1]):
    if np.std(y_test[:, i]) > 1e-6 and np.std(y_pred[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test[:, i], y_pred[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
    worksheet.update(
    range_name='E4',
    values=[[f"{average_correlation:.4f}"]]
    )
else:
    print("  No correlations could be calculated for averaging.")

correlation_string = ', '.join([f"{c:.4f}" for c in correlations])
# Update cell E3 with the full string
worksheet.update('E3', [[correlation_string]])

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence = r2_score(y_test[:, 0], y_pred[:, 0])
print("  valence =", r2_valence)

r2_tension = r2_score(y_test[:, 1], y_pred[:, 1])
print("  tension =", r2_tension)

r2_energy = r2_score(y_test[:, 2], y_pred[:, 2])
print("  energy =", r2_energy)

r2_like = r2_score(y_test[:, 3], y_pred[:, 3])
print("  like =", r2_like)

worksheet.update(
    range_name='E7',
    values=[[f"{r2_valence:.4f}, {r2_tension:.4f}, {r2_energy:.4f}, {r2_like:.4f}"]]
)



Shape of predictions (y_pred): (767, 4)
First 5 actual values (y_test):
[[3.96 8.32 7.05 8.  ]
 [7.11 7.01 8.11 7.01]
 [4.68 5.04 5.88 5.1 ]
 [2.08 4.04 7.6  1.26]
 [6.34 6.43 5.32 4.8 ]]
First 5 predicted values (y_pred):
[[5.47911   5.934202  5.1065726 5.603344 ]
 [4.55486   5.023981  4.6518345 5.032511 ]
 [3.4750135 2.8988795 6.6237874 3.0516129]
 [3.012626  2.276279  7.0963893 2.5203476]
 [5.050826  4.791978  6.146352  4.718383 ]]



/tmp/ipython-input-26-2279673651.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E5', [[f"{mape:.2f}%"]])


Mean Absolute Percentage Error (MAPE): 42.14%

Mean Absolute Error (MAE): 1.3936


/tmp/ipython-input-26-2279673651.py:20: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E2', [[f"{mae:.4f}"]])
/tmp/ipython-input-26-2279673651.py:25: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E6', [[f"{rmse:.4f}"]])


Root Mean Squared Error (RMSE): 1.7477

Pearson Correlation Coefficients (per dimension):
  Positive Dimension: 0.5505
  Relaxed Dimension: 0.6048
  Awake Dimension: 0.2551
  Like Dimension: 0.5590
  Average Pearson Correlation across dimensions: 0.4924


/tmp/ipython-input-26-2279673651.py:54: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E3', [[correlation_string]])



R-squared scores:
  valence = 0.3024829635282993
  tension = 0.3654707476151795
  energy = 0.05967284128173378
  like = 0.3110635695126158


{'spreadsheetId': '1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8',
 'updatedRange': "'htsat-fused'!E7",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_dim_p = mlp_regressor_dim_p.predict(X_test_dim_p)

print(f"\nShape of predictions (y_pred): {y_pred_dim_p.shape}")
print(f"First 5 actual values (y_test):\n{y_test_dim_p[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_dim_p[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error = np.abs((y_test_dim_p - y_pred_dim_p) / y_test_dim_p) * 100
mape_dim_p = np.mean(absolute_percentage_error)
worksheet.update('E12', [[f"{mape_dim_p:.2f}%"]])
print(f"Mean Absolute Percentage Error (MAPE): {mape_dim_p:.2f}%\n")

# Mean Absolute Error (MAE)
mae_dim_p = mean_absolute_error(y_test_dim_p, y_pred_dim_p)
worksheet.update('E9', [[f"{mae_dim_p:.4f}"]])
print(f"Mean Absolute Error (MAE): {mae_dim_p:.4f}")

# Root Mean Squared Error (RMSE)
rmse_dim_p = np.sqrt(mean_squared_error(y_test_dim_p, y_pred_dim_p))
worksheet.update('E13', [[f"{rmse_dim_p:.4f}"]])
print(f"Root Mean Squared Error (RMSE): {rmse_dim_p:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Positive', 'Relaxed', 'Awake', 'Like']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_dim_p[:, i]) > 1e-6 and np.std(y_pred_dim_p[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_dim_p[:, i], y_pred_dim_p[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_dim_p.shape[1]):
    if np.std(y_test_dim_p[:, i]) > 1e-6 and np.std(y_pred_dim_p[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_dim_p[:, i], y_pred_dim_p[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    worksheet.update(
    range_name='E11',
    values=[[f"{average_correlation:.4f}"]]
    )
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

correlation_string_dim_p = ', '.join([f"{c:.4f}" for c in correlations])
# Update cell E3 with the full string
worksheet.update('E10', [[correlation_string_dim_p]])

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence_dim_p = r2_score(y_test_dim_p[:, 0], y_pred_dim_p[:, 0])
print("  valence =", r2_valence_dim_p)

r2_tension_dim_p = r2_score(y_test_dim_p[:, 1], y_pred_dim_p[:, 1])
print("  tension =", r2_tension_dim_p)

r2_energy_dim_p = r2_score(y_test_dim_p[:, 2], y_pred_dim_p[:, 2])
print("  energy =", r2_energy_dim_p)

r2_like_dim_p = r2_score(y_test_dim_p[:, 3], y_pred_dim_p[:, 3])
print("  like =", r2_like_dim_p)

worksheet.update(
    range_name='E14',
    values=[[f"{r2_valence_dim_p:.4f}, {r2_tension_dim_p:.4f}, {r2_energy_dim_p:.4f}, {r2_like_dim_p:.4f}"]]
)


Shape of predictions (y_pred): (791, 4)
First 5 actual values (y_test):
[[3.16 4.31 2.02 5.18]
 [8.86 8.86 8.78 8.53]
 [8.14 5.35 3.35 7.12]
 [5.02 5.01 5.99 5.98]
 [5.04 4.31 6.31 5.45]]
First 5 predicted values (y_pred):
[[4.68286   3.8668928 6.586182  3.930443 ]
 [6.7634263 5.8531747 5.9852066 6.54536  ]
 [4.375108  3.2750263 6.990559  3.6191962]
 [5.3131275 5.0134277 5.8963675 5.1165404]
 [3.305712  3.7067304 3.7343602 4.0418177]]

Mean Absolute Percentage Error (MAPE): 46.03%



/tmp/ipython-input-27-2848619493.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E12', [[f"{mape_dim_p:.2f}%"]])
/tmp/ipython-input-27-2848619493.py:20: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E9', [[f"{mae_dim_p:.4f}"]])


Mean Absolute Error (MAE): 1.4689
Root Mean Squared Error (RMSE): 1.8336

Pearson Correlation Coefficients (per dimension):
  Positive Dimension: 0.5667
  Relaxed Dimension: 0.5277
  Awake Dimension: 0.4375
  Like Dimension: 0.5835


/tmp/ipython-input-27-2848619493.py:25: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E13', [[f"{rmse_dim_p:.4f}"]])


  Average Pearson Correlation across dimensions: 0.5288

R-squared scores:
  valence = 0.32106427291368544
  tension = 0.2766442146574206
  energy = 0.18715132125494782
  like = 0.3388074426537657


/tmp/ipython-input-27-2848619493.py:54: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E10', [[correlation_string_dim_p]])


{'spreadsheetId': '1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8',
 'updatedRange': "'htsat-fused'!E14",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [28]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_disc_i = mlp_regressor_disc_i.predict(X_test_disc_i)

print(f"\nShape of predictions (y_pred): {y_pred_disc_i.shape}")
print(f"First 5 actual values (y_test):\n{y_test_disc_i[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_disc_i[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error_disc_i = np.abs((y_test_disc_i - y_pred_disc_i) / y_test_disc_i) * 100
mape_disc_i = np.mean(absolute_percentage_error_disc_i)
worksheet.update('E19', [[f"{mape_disc_i:.2f}%"]])
print(f"Mean Absolute Percentage Error (MAPE): {mape_disc_i:.2f}%\n")

# Mean Absolute Error (MAE)
mae_disc_i = mean_absolute_error(y_test_disc_i, y_pred_disc_i)
worksheet.update('E16', [[f"{mae_disc_i:.4f}"]])
print(f"Mean Absolute Error (MAE): {mae_disc_i:.4f}")

# Root Mean Squared Error (RMSE)
rmse_disc_i = np.sqrt(mean_squared_error(y_test_disc_i, y_pred_disc_i))
worksheet.update('E20', [[f"{rmse_disc_i:.4f}"]])
print(f"Root Mean Squared Error (RMSE): {rmse_disc_i:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Happiness', 'Sadness', 'Anger', 'Tenderness', 'Fear']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_disc_i[:, i]) > 1e-6 and np.std(y_pred_disc_i[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_disc_i[:, i], y_pred_disc_i[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_disc_i.shape[1]):
    if np.std(y_test_disc_i[:, i]) > 1e-6 and np.std(y_pred_disc_i[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_disc_i[:, i], y_pred_disc_i[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    worksheet.update(
        range_name='E18',
        values=[[f"{average_correlation:.4f}"]]
      )
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

correlation_string_disc_i = ', '.join([f"{c:.4f}" for c in correlations])
# Update cell E3 with the full string
worksheet.update('E17', [[correlation_string_disc_i]])

from sklearn.metrics import r2_score

# R-squared IDisc
print("\nR-squared scores (IDisc):")
r2_happiness_disc_i = r2_score(y_test_disc_i[:, 0], y_pred_disc_i[:, 0])
print("  happiness =", r2_happiness_disc_i)
r2_sadness_disc_i = r2_score(y_test_disc_i[:, 1], y_pred_disc_i[:, 1])
print("  sadness =", r2_sadness_disc_i)
r2_anger_disc_i = r2_score(y_test_disc_i[:, 2], y_pred_disc_i[:, 2])
print("  anger =", r2_anger_disc_i)
r2_tenderness_disc_i = r2_score(y_test_disc_i[:, 3], y_pred_disc_i[:, 3])
print("  tenderness =", r2_tenderness_disc_i)
r2_fear_disc_i = r2_score(y_test_disc_i[:, 4], y_pred_disc_i[:, 4])
print("  fear =", r2_fear_disc_i)

worksheet.update(
    range_name='E21',
    values=[[f"{r2_happiness_disc_i:.4f}, {r2_sadness_disc_i:.4f}, {r2_anger_disc_i:.4f}, {r2_tenderness_disc_i:.4f}, {r2_fear_disc_i:.4f}"]]
)


Shape of predictions (y_pred): (779, 5)
First 5 actual values (y_test):
[[1.   1.   1.   1.   1.  ]
 [4.23 6.82 5.75 5.13 5.69]
 [1.28 1.24 1.15 1.18 1.23]
 [1.   1.   2.4  1.   1.54]
 [1.36 4.73 1.07 3.37 1.94]]
First 5 predicted values (y_pred):
[[1.7944102 3.3382258 3.4380221 2.5198338 2.851096 ]
 [2.0899372 2.8112595 3.7990627 2.5752058 2.7045498]
 [3.127357  1.9672538 1.375336  3.0326624 2.2028012]
 [2.060343  3.1218722 4.956721  2.3910613 3.9279404]
 [2.3352883 2.404929  1.8917665 2.8472476 1.5811555]]

Mean Absolute Percentage Error (MAPE): 82.91%



/tmp/ipython-input-28-609622067.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E19', [[f"{mape_disc_i:.2f}%"]])
/tmp/ipython-input-28-609622067.py:20: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E16', [[f"{mae_disc_i:.4f}"]])


Mean Absolute Error (MAE): 1.6023
Root Mean Squared Error (RMSE): 2.0451

Pearson Correlation Coefficients (per dimension):
  Happiness Dimension: 0.3904
  Sadness Dimension: 0.2074
  Anger Dimension: 0.4880
  Tenderness Dimension: 0.2629
  Fear Dimension: 0.2819


/tmp/ipython-input-28-609622067.py:25: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E20', [[f"{rmse_disc_i:.4f}"]])


  Average Pearson Correlation across dimensions: 0.3261

R-squared scores (IDisc):
  happiness = 0.14956115509667667
  sadness = 0.03733611859295083
  anger = 0.23260933875390033
  tenderness = 0.0646636825669632
  fear = 0.07292944774456644


/tmp/ipython-input-28-609622067.py:54: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E17', [[correlation_string_disc_i]])


{'spreadsheetId': '1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8',
 'updatedRange': "'htsat-fused'!E21",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_disc_p = mlp_regressor_disc_p.predict(X_test_disc_p)

print(f"\nShape of predictions (y_pred): {y_pred_disc_p.shape}")
print(f"First 5 actual values (y_test):\n{y_test_disc_p[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_disc_p[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error_disc_p = np.abs((y_test_disc_p - y_pred_disc_p) / y_test_disc_p) * 100
mape_disc_p = np.mean(absolute_percentage_error_disc_p)
worksheet.update('E26', [[f"{mape_disc_p:.2f}%"]])
print(f"Mean Absolute Percentage Error (MAPE): {mape_disc_p:.2f}%\n")

# Mean Absolute Error (MAE)
mae_disc_p = mean_absolute_error(y_test_disc_p, y_pred_disc_p)
worksheet.update('E23', [[f"{mae_disc_p:.4f}"]])
print(f"Mean Absolute Error (MAE): {mae_disc_p:.4f}")

# Root Mean Squared Error (RMSE)
rmse_disc_p = np.sqrt(mean_squared_error(y_test_disc_p, y_pred_disc_p))
worksheet.update('E27', [[f"{rmse_disc_p:.4f}"]])
print(f"Root Mean Squared Error (RMSE): {rmse_disc_p:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Happiness', 'Sadness', 'Anger', 'Tenderness', 'Fear']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_disc_p[:, i]) > 1e-6 and np.std(y_pred_disc_p[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_disc_p[:, i], y_pred_disc_p[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_disc_p.shape[1]):
    if np.std(y_test_disc_p[:, i]) > 1e-6 and np.std(y_pred_disc_p[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_disc_p[:, i], y_pred_disc_p[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    worksheet.update(
        range_name='E25',
        values=[[f"{average_correlation:.4f}"]]
      )
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

correlation_string_disc_p = ', '.join([f"{c:.4f}" for c in correlations])
# Update cell E3 with the full string
worksheet.update('E24', [[correlation_string_disc_p]])

from sklearn.metrics import r2_score

# R-squared PDisc
print("\nR-squared scores (PDisc):")
r2_happiness_disc_p = r2_score(y_test_disc_p[:, 0], y_pred_disc_p[:, 0])
print("  happiness =", r2_happiness_disc_p)
r2_sadness_disc_p = r2_score(y_test_disc_p[:, 1], y_pred_disc_p[:, 1])
print("  sadness =", r2_sadness_disc_p)
r2_anger_disc_p = r2_score(y_test_disc_p[:, 2], y_pred_disc_p[:, 2])
print("  anger =", r2_anger_disc_p)
r2_tenderness_disc_p = r2_score(y_test_disc_p[:, 3], y_pred_disc_p[:, 3])
print("  tenderness =", r2_tenderness_disc_p)
r2_fear_disc_p = r2_score(y_test_disc_p[:, 4], y_pred_disc_p[:, 4])
print("  fear =", r2_fear_disc_p)

worksheet.update(
    range_name='E28',
    values=[[f"{r2_happiness_disc_p:.4f}, {r2_sadness_disc_p:.4f}, {r2_anger_disc_p:.4f}, {r2_tenderness_disc_p:.4f}, {r2_fear_disc_p:.4f}"]]
)

average_r2 = (r2_valence + r2_energy + r2_tension) /3
worksheet.update('E8', [[f"{average_r2:.4f}"]])

average_r2_dim_p = (r2_valence_dim_p + r2_energy_dim_p + r2_tension_dim_p) /3
worksheet.update('E15', [[f"{average_r2_dim_p:.4f}"]])

average_r2_disc_i = (r2_happiness_disc_i + r2_sadness_disc_i + r2_anger_disc_i + r2_tenderness_disc_i + r2_fear_disc_i) / 5
worksheet.update('E22', [[f"{average_r2_disc_i:.4f}"]])

average_r2_disc_p = (r2_happiness_disc_p + r2_sadness_disc_p + r2_anger_disc_p + r2_tenderness_disc_p + r2_fear_disc_p) / 5
worksheet.update('E29', [[f"{average_r2_disc_p:.4f}"]])

print("\nAverage R-squared scores:")
print("IDim: ", average_r2)
print("PDim: ", average_r2_dim_p)
print("IDisc: ", average_r2_disc_i)
print("PDisc: ", average_r2_disc_p)


Shape of predictions (y_pred): (767, 5)
First 5 actual values (y_test):
[[2.   6.04 1.86 1.01 1.01]
 [1.   6.06 1.   2.99 1.  ]
 [1.   2.01 4.02 1.   1.  ]
 [1.81 5.81 4.99 3.35 1.61]
 [1.26 3.29 2.45 1.26 6.45]]
First 5 predicted values (y_pred):
[[1.9828969 5.0825434 2.939643  3.0000086 3.0411415]
 [1.3349642 4.4660406 4.8901353 2.1934712 4.3548226]
 [2.100566  4.347086  4.2183466 2.6652856 4.5488334]
 [2.154156  3.9351027 4.8210325 2.4145894 5.2276273]
 [2.607668  4.7895174 2.552754  3.5414739 3.3412185]]

Mean Absolute Percentage Error (MAPE): 87.64%



/tmp/ipython-input-29-1059293014.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E26', [[f"{mape_disc_p:.2f}%"]])
/tmp/ipython-input-29-1059293014.py:20: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E23', [[f"{mae_disc_p:.4f}"]])


Mean Absolute Error (MAE): 1.7959
Root Mean Squared Error (RMSE): 2.1997

Pearson Correlation Coefficients (per dimension):
  Happiness Dimension: 0.3992
  Sadness Dimension: 0.3627
  Anger Dimension: 0.4239
  Tenderness Dimension: 0.3272
  Fear Dimension: 0.4040


/tmp/ipython-input-29-1059293014.py:25: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E27', [[f"{rmse_disc_p:.4f}"]])


  Average Pearson Correlation across dimensions: 0.3834

R-squared scores (PDisc):
  happiness = 0.14354068581492307
  sadness = 0.13065007263295836
  anger = 0.1664675284034206
  tenderness = 0.10424823390432825
  fear = 0.15755359247838796


/tmp/ipython-input-29-1059293014.py:54: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E24', [[correlation_string_disc_p]])
/tmp/ipython-input-29-1059293014.py:77: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E8', [[f"{average_r2:.4f}"]])
/tmp/ipython-input-29-1059293014.py:80: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E15', [[f"{average_r2_dim_p:.4f}"]])
/tmp/ipython-input-29-1059293014.py:83: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, v


Average R-squared scores:
IDim:  0.24254218414173753
PDim:  0.2616199362753513
IDisc:  0.11141994855101149
PDisc:  0.14049202264680366
